In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxopt
from numpy import linalg

In [121]:
# Creates the kernel matrix
def create_kernel_mat(X,kernel,gamma):
    print(1)
    
    if kernel == 'linear': 
        return X @ X.T
    elif kernel == 'rbf':
        X_norm = np.sum(X**2,axis=1).reshape(-1,1) #calculates the squared norm of every data point
        sq_dist = X_norm + X_norm.T -2*(X @ X.T)
        return np.exp(gamma * sq_dist)

In [82]:
# Finds the optimal values for the lagrange multipliers in the dual problem
def QPoptimize(kernel_matrix,y,C,n_samples):
    
    #converting to cvxopt format
    P = cvxopt.matrix(np.outer(y,y) * kernel_matrix,tc='d')
    q = cvxopt.matrix(-np.ones((n_samples, 1)))
    A = cvxopt.matrix(y,(1,n_samples),tc='d')
    b = cvxopt_matrix(np.zeros(1))
    
    if C is None:
        G = cvxopt_matrix(-np.eye(n_samples))
        h = cvxopt_matrix(np.zeros(n_samples))
    else:
        G = cvxopt_matrix(np.vstack((-np.eye(n_samples),np.eye(n_samples))))
        h = cvxopt_matrix(np.vstack((np.zeros((n_samples,1)),np.ones((n_samples,1))*C)))
        
    #Setting solver parameters (change default to decrease tolerance) 
    cvxopt_solvers.options['show_progress'] = False
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10
    
    #Run solver
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(sol['x'])
    
    return alphas

In [106]:
def train_SVC(X,y,kernel_matrix,C):
    n_samples = X.shape[0]
    
    alphas = QPoptimize(kernel_matrix,y,C,n_samples)
    
    ind = np.arange(n_samples)[alphas > 1e-4] #Indices where the multipliers are non zero
    support_vecs = X[ind] #Support vecs are the vectors corresponding to non zero indices
    support_multipliers = alphas[ind]
    support_labels = y[ind]
    
    b = y[ind[0]] - np.sum(y * alphas * K[ind[0]].reshape(-1,1)) #b can be computed using only one of the support vectors
    
    return support_vecs,support_multipliers,support_labels,b

In [107]:
class SVC:
    def __init__(self,kernel='rbf',C=1,gamma=0.5,degree=3):
        self.kernel = kernel
        self.C = float(C)
        self.gamma = gamma
        self.degree = degree

    def fit(self,X,y):
        kernel_matrix = create_kernel_mat(X,kernel=self.kernel,gamma=self.gamma)
        self.support_vecs_,self.support_multipliers_,self.support_labels,self.b = train_SVC(X,y,kernel_matrix,C=self.C)

In [108]:
clf = SVC()

In [109]:
clf.fit(a,y)

ValueError: Rank(A) < p or Rank([P; A; G]) < n

In [123]:
class SupportVectorMachine(object):
    """The Support Vector Machine classifier.
    Uses cvxopt to solve the quadratic optimization problem.
    Parameters:
    -----------
    C: float
        Penalty term.
    kernel: function
        Kernel function. Can be either polynomial, rbf or linear.
    power: int
        The degree of the polynomial kernel. Will be ignored by the other
        kernel functions.
    gamma: float
        Used in the rbf kernel function.
    coef: float
        Bias term used in the polynomial kernel function.
    """
    def __init__(self, C=1, kernel=rbf_kernel, power=4, gamma=None, coef=4):
        self.C = C
        self.kernel = kernel
        self.power = power
        self.gamma = gamma
        self.coef = coef
        self.lagr_multipliers = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.intercept = None

    def fit(self, X, y):

        n_samples, n_features = np.shape(X)

        # Set gamma to 1/n_features by default
        if not self.gamma:
            self.gamma = 1 / n_features


        # Calculate kernel matrix
        kernel_matrix = create_kernel_mat(X,kernel='rbf',gamma=self.gamma)
        print(kernel_matrix)
        # Define the quadratic optimization problem
        P = cvxopt.matrix(np.outer(y, y) * kernel_matrix, tc='d')
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1, n_samples), tc='d')
        b = cvxopt.matrix(0, tc='d')

        if not self.C:
            G = cvxopt.matrix(np.identity(n_samples) * -1)
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            G_max = np.identity(n_samples) * -1
            G_min = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((G_max, G_min)))
            h_max = cvxopt.matrix(np.zeros(n_samples))
            h_min = cvxopt.matrix(np.ones(n_samples) * self.C)
            h = cvxopt.matrix(np.vstack((h_max, h_min)))

        # Solve the quadratic optimization problem using cvxopt
        minimization = cvxopt.solvers.qp(P, q, G, h, A, b)

        # Lagrange multipliers
        lagr_mult = np.ravel(minimization['x'])

        # Extract support vectors
        # Get indexes of non-zero lagr. multipiers
        idx = lagr_mult > 1e-7
        # Get the corresponding lagr. multipliers
        self.lagr_multipliers = lagr_mult[idx]
        # Get the samples that will act as support vectors
        self.support_vectors = X[idx]
        # Get the corresponding labels
        self.support_vector_labels = y[idx]

        # Calculate intercept with first support vector
        self.intercept = self.support_vector_labels[0]
        for i in range(len(self.lagr_multipliers)):
            self.intercept -= self.lagr_multipliers[i] * self.support_vector_labels[
                i] * self.kernel(self.support_vectors[i], self.support_vectors[0])

    def predict(self, X):
        y_pred = []
        # Iterate through list of samples and make predictions
        for sample in X:
            prediction = 0
            # Determine the label of the sample by the support vectors
            for i in range(len(self.lagr_multipliers)):
                prediction += self.lagr_multipliers[i] * self.support_vector_labels[
                    i] * self.kernel(self.support_vectors[i], sample)
            prediction += self.intercept
            y_pred.append(np.sign(prediction))
        return np.array(y_pred)

In [115]:
y = np.array([1,0,0,1,1]).reshape(-1,1)

In [124]:
clf2 = SupportVectorMachine()

In [125]:
clf2.fit(a,y)

1
[[1.00000000e+00 1.48413159e+02 1.48413159e+02 2.68337287e+05
  8.10308393e+03]
 [1.48413159e+02 1.00000000e+00 1.00000000e+00 9.00171313e+01
  7.38905610e+00]
 [1.48413159e+02 1.00000000e+00 1.00000000e+00 9.00171313e+01
  7.38905610e+00]
 [2.68337287e+05 9.00171313e+01 9.00171313e+01 1.00000000e+00
  1.64872127e+00]
 [8.10308393e+03 7.38905610e+00 7.38905610e+00 1.64872127e+00
  1.00000000e+00]]


ValueError: Rank(A) < p or Rank([P; A; G]) < n